# Demo notebook for slide 3.4

In [1]:
%matplotlib widget

In [2]:
import bmcs_utils

In [3]:
from ibvpy.tfunction import TimeFunction, TFSelector, TFCyclicSin

ImportError: cannot import name 'TimeFunction' from 'ibvpy.tfunction' (C:\Users\abdra\miniconda3\envs\bmcs_env\lib\site-packages\ibvpy\tfunction\__init__.py)

In [4]:
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import bmcs_utils.api as bu

ImportError: cannot import name 'TimeFunction' from 'ibvpy.tfunction' (C:\Users\abdra\miniconda3\envs\bmcs_env\lib\site-packages\ibvpy\tfunction\__init__.py)

In [5]:
se = SlideExplorer()

In [6]:
#sm = se.slide_model.C_codegen()

Parameters for elementary convergence studies - no physical relevance

Parameters rendering a stress-strain response comparable with a `MARS` model - example concrete

In [7]:
material_params = dict(
    E_T=1, gamma_T=1, K_T=1, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=5, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.5
)

Critical step size - step smaller than the elastic range in tension.

In [15]:
se = SlideExplorer(n_steps=2000, k_max=50)
se.slide_model.trait_set(**material_params)
se.tf_s_x = TFCyclicSin(number_of_cycles = 10, t_max = 1)
#se.tf_s_y = TFCyclicSin(number_of_cycles = 40, t_max = 1, phase_shift=0.25)

In [16]:
se.slide_model.trait_set(**material_params)
se.trait_set(s_x_1 = 3.1, s_y_1 = 2, w_1 = 0);
se.run()
# se.trait_set(s_x_1 = 2, s_y_1 = 1, w_1 = 0);
# se.run()
# se.trait_set(s_x_1 = 10, s_y_1 = 0, w_1 = 10);
# se.run()

In [17]:
se.energy_dissipation.show_iter = False
se.inel_state_evolution.t_max=3.0
se.interact()

In [14]:
import numpy as np
material_params = dict(
    E_T=1, gamma_T=0, K_T=1, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=5, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.5
)
eta_range = np.linspace(0,1,5)
se_T_list = []
se_N_list = []
#for m in [1e-9, 0.05]:
for eta in eta_range:
    material_params['eta'] = eta
    se1 = SlideExplorer(n_steps=100, k_max=50)
    se1.slide_model.trait_set(**material_params)
#     se1.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = 0.8);
#     se1.run()
    se1.trait_set(s_x_1 = 20, s_y_1 = 0, w_1 = 0);
    se1.run()
    se_T_list.append(se1)
    se2 = SlideExplorer(n_steps=200, k_max=50)
    se2.slide_model.trait_set(**material_params)
    se2.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = 20);
    se2.run()
    se_N_list.append(se2)

In [15]:
import matplotlib.pylab as plt
fig, (ax_T, ax_N) = plt.subplots(1,2, tight_layout=True, figsize=(10,4))
fig.canvas.header_visible=False
ax_T.plot([0,1],[0,1],'--',color='black')
ax_N.plot([0,1],[0,1],'--',color='black')
for se_T, se_N in zip(se_T_list, se_N_list):
    s_x_pi_, s_y_pi_, w_pi_, z_, alpha_x_, alpha_y_, omega_T_, omega_N_ = se_N.Eps_arr.T
    #ax_T.plot(omega_T_,omega_N_)
    se_T.inel_state_evolution.plot_omega_NT(ax_T)
    se_N.inel_state_evolution.plot_omega_NT(ax_N)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
fig.savefig('myfig.pdf')

In [16]:
se_T_list[1].inel_state_evolution.interact()

In [17]:
import sympy as sp
S_N, S_T, eta = sp.symbols('S_N, S_T, eta')
Y_N, Y_T = sp.symbols('Y_N, Y_T')
omega_N, omega_T = sp.symbols('omega_N, omega_T')
c_N, c_T = sp.symbols('c_N, c_T')

phi_N_ext = ( # (1-omega_N)**c_N * (
    (Y_N**2 + eta * (Y_T * Y_N)) /
    (2*(S_N - eta * (S_N - sp.sqrt(S_N * S_T))))
)
phi_T_ext = ( # (1-omega_T)**c_T * (
    (Y_T**2 + eta * (Y_T * Y_N)) /
    (2*(S_T - eta * (S_T - sp.sqrt(S_N * S_T))))
)
phi_ext = phi_N_ext + phi_T_ext

In [18]:
sp.simplify(phi_ext.diff(Y_N).subs(eta,1))

(Y_N + Y_T)/sqrt(S_N*S_T)

In [19]:
sp.simplify(phi_ext.diff(Y_T).subs(eta,1))

(Y_N + Y_T)/sqrt(S_N*S_T)

In [20]:
sp.simplify(phi_ext)

(Y_N*(S_T - eta*(S_T - sqrt(S_N*S_T)))*(Y_N + Y_T*eta) + Y_T*(S_N - eta*(S_N - sqrt(S_N*S_T)))*(Y_N*eta + Y_T))/(2*(S_N - eta*(S_N - sqrt(S_N*S_T)))*(S_T - eta*(S_T - sqrt(S_N*S_T))))

In [21]:
import sympy as sp
E_, K_ = sp.symbols('E, K')
get_EK = sp.lambdify( (E_,K_) , E_*K_ / (E_+K_) - E_ )
_, ax_EK = plt.subplots(1,1)
K_r = np.linspace(1,100,100)
ax_EK.plot(K_r, get_EK(1,K_r))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Debugging support

You can access the values of the state variables and of the the associated 
thermodynamic forces by accessing the `Sig_t` and `Eps_t` arrays.
Each row represents the time step and each column a state variable.
The order of state variables corresponds to the order in the symbolic 
object `se.slide_model.symb`

In [22]:
se.Eps_t[-1,:], se.Sig_t[-1,:]

(array([-0.02841641, -0.31802875,  0.01197538,  1.19753845,  0.10272165,
         0.13486184,  0.90182281,  0.50622116]),
 array([ 2.78984374e-03, -3.61485598e-01, -1.19753835e-02,  1.19753845e+00,
         1.02721649e-01,  1.34861836e-01,  6.77885988e+00,  0.00000000e+00]))

In [23]:
se.Sig_arr[-1,:]

array([ 2.78984374e-03, -3.61485598e-01, -1.19753835e-02,  1.19753845e+00,
        1.02721649e-01,  1.34861836e-01,  6.77885988e+00,  0.00000000e+00])